In [13]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm # progress apply :)
tqdm.pandas()

%load_ext autoreload
%autoreload 2
def d(l):
    pd.set_option('display.max_colwidth', l)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# just to check if new articles are included
new_articles=pd.read_excel("2025-04-26 Manual Annotation results_Inclusion.xlsx")["ID"].to_list()

In [15]:
df_country_fields=pd.read_json("2025-06-07 Country_fields results.json")
df_dict=pd.read_json('2025-06-07 data with dict_approach.json')
df_GPT4=pd.read_json("2025-06-08 full data with 6x GPT41.json")

In [16]:
df_GPT4[df_GPT4.name=="Nature1223"].category

94    Mistake
Name: category, dtype: object

In [17]:
# Check if also the new articles are in the processed collections
print("GPT files",df_GPT4[df_GPT4.name.isin(new_articles)].shape[0],len(new_articles))
print("dict approach",df_dict[df_dict.name.isin(new_articles)].shape[0],len(new_articles))
print("country fields",df_country_fields[df_country_fields.name.isin(new_articles)].shape[0],len(new_articles))

GPT files 17 17
dict approach 17 17
country fields 17 17


In [18]:
df_meta_data=pd.read_excel("2025-06-07 all meta data.xlsx")[['ID', 'Doi', 'journal', 'Title', 'Date', 'Year', 'Type',
       'Type Recoded', 'confirmed_2024', '9. retained for analysis (revised)']]

df_meta_data["name"]=df_meta_data.ID
df_meta_data.index=df_meta_data.ID
#df_meta_data.rename(columns={"ID": "name","Doi_x": "Doi"},inplace=True)

print(df_meta_data.shape)
df_meta_data.head()

(3231, 11)


,ID,Doi,journal,Title,Date,Year,Type,Type Recoded,confirmed_2024,9. retained for analysis (revised),name
ID,,,,,,,,,,,
Nature1,Nature1,10.1038/d41586-023-01969-z,Nature,Prized dinosaur fossil returned to Brazil afte...,2023-06-22,2023,News,Journalism,True,True,Nature1
Nature10,Nature10,10.1038/d41586-023-01416-z,Nature,Fish on dry land hint at why we blink,2023-04-27,2023,Research Highlights,Other,False,False,Nature10
Nature100,Nature100,10.1038/d41586-021-01457-2,Nature,Cells or drugs? The race to regenerate the heart,2021-06-09,2021,Comments & Opinion,Journalism,True,True,Nature100
Nature1000,Nature1000,10.1038/118855a0,Nature,Research Items,1926-12-11,1926,News,Contribution,False,False,Nature1000
Nature1001,Nature1001,10.1038/news050926-10,Nature,Water walkers surf on the edge,2005-09-28,2005,News,Contribution,False,False,Nature1001


In [19]:
# Start with the first DataFrame

df_merged = df_meta_data

# Iteratively merge the remaining DataFrames on the "name" column
for df in [df_country_fields ,df_dict,df_GPT4]:
    df_merged = pd.merge(df_merged, df, on="name", how="outer",suffixes=(None, '_y'))
# df_merged[df_merged["Year"].isna()][df_old.columns].to_excel("2025_02_04 missing_date_DOI_Type_recoded.xlsx")

df.shape

(1018, 35)

In [20]:
#df_merged.rename(columns={"text_x":"text"},inplace=True)
df=df_merged[df_merged["9. retained for analysis (revised)"]]

In [21]:
df.shape

(1016, 74)

In [22]:
df["category"].value_counts()

category
Misconduct    626
Mistake       230
None          160
Name: count, dtype: int64

# Add manual Annotation (classification)

In [23]:
#df.loc[(df["category"]!="Misconduct"),["name","retraction_text","text",'category',"explanation"]].to_excel("2025-06-06 Mistake_none Sample for manual annotation.xlsx")

In [24]:
import pandas as pd

# List of file paths and corresponding dataset numbers
files = [
    ("2025-02-24 Sample for manual annotation_filled.xlsx", 1),
    ("2025-03-07 Sample for manual annotation filled.xlsx", 2),
    ("2025-03-07 2nd Sample for manual annotation filled.xlsx", 3),
    ("2025-06-06 none_mistake sample ANNOTATED.xlsx", 4),
    ("2025-06-08 Sample Auste Annotated 2024-03-19.xlsx", 5),


]

dfs = []
for filepath, num in files:
    dfa = (
        pd.read_excel(filepath)[["name", "Max code"]]
        .dropna(subset=["Max code"])
        .set_index("name")
    )
    dfa["Max code"] = dfa["Max code"].str.lower()
    dfa.rename(columns={"Max code": f"Max code_{num}"}, inplace=True)
    
    # Optional: remove duplicated index entries within each dataframe, keeping last
    dfa = dfa[~dfa.index.duplicated(keep='last')]
    
    dfs.append(dfa)

# Concatenate all dataframes along columns, aligning on index 'name'
df_annotated = pd.concat(dfs, axis=1)

# Identify rows where values differ across datasets
df_annotated["all_equal"] = df_annotated.nunique(axis=1) == 1
df_annotated["counts"] = df_annotated.drop(columns=["all_equal"]).count(axis=1)

# Filter rows with differences
df_differences = df_annotated[~df_annotated["all_equal"]].drop(columns=["all_equal", "counts"])

# Create a new column "Max code" that takes the first non-NaN value from the Max code columns
max_code_columns = [col for col in df_annotated.columns if col.startswith('Max code_')]
df_annotated['Max code'] = df_annotated[max_code_columns].bfill(axis=1).iloc[:, 0]

# Example: count of counts values
print(df_annotated.counts.value_counts())


counts
1    191
2     18
3      3
Name: count, dtype: int64


KeyError: ('Max code', 'category', 'dict-topic')

In [134]:

#df_final.dropna(subset="category")

df_final = df.merge(df_annotated, left_on="name",right_on="name", how="left")
df_final.shape

(1016, 82)

In [135]:
# Drop all columns with '_y' in their names and remove duplicate columns
df_final = df_final.loc[:, ~df_final.columns.duplicated()]  # Remove duplicate columns
df_final = df_final.drop(columns=[col for col in df.columns if '_y' in col])  # Drop columns with '_y'


In [136]:
#df_final=df_final.dropna(subset="Year")
df_final.sort_values("name").to_json("2025-06-06 Merged_Data.json", date_format='iso')
df_final.sort_values("name").to_excel("2025-06-06 Merged_Data.xlsx")

In [137]:
df_final.columns

Index(['ID', 'Doi', 'journal', 'Title', 'Date', 'Year', 'Type', 'Type Recoded',
       'confirmed_2024', '9. retained for analysis (revised)', 'name',
       'contains_country', 'country_plus_city', 'Most Common country',
       'Most Common country_no_city', 'all_jobs', 'fields',
       'most_common_fields', 'path', 'folder', 'labels', 'retraction_counts',
       'text', 'sentiment', 'keywords_likely', 'keywords_only',
       'keywords_filtered', 'Lemmata', 'NoStopwords', 'Nouns_Verbs',
       'ner_orga', 'ner_person', 'misconduct', 'mistake', 'retract',
       'n_misconduct', 'n_mistake', 'n_retract', 'dict_topic', 'category_1',
       'explanation_1', 'original_index', 'name_2', 'category_2',
       'explanation_2', 'country_1', 'field_1', 'name_3', 'category_3',
       'explanation_3', 'country_3', 'field_3', 'name_4', 'category_4',
       'explanation_4', 'country_4', 'field_4', 'name_5', 'category_5',
       'explanation_5', 'country_5', 'field_5', 'name_6', 'category_6',
       

In [138]:
df_final[df_final.text.isna()]

,ID,Doi,journal,Title,Date,Year,Type,Type Recoded,confirmed_2024,9. retained for analysis (revised),...,category,difference_count,Max code_1,Max code_2,Max code_3,Max code_4,Max code_5,all_equal,counts,Max code
